**Mapping**

ETM offers quite some granularity in the returned hourly carrier curves which
depending on the usecase might not always be desirable. The default keys can
simply be replaced and aggregated to an user-defined naming convention.

In [1]:
from pyetm import Client
import pandas as pd

# get electricity curves
client = Client(1008593)
ecurves = client.get_hourly_electricity_curves()

# show head
ecurves.head()

,agriculture_chp_engine_biogas.output (MW),agriculture_chp_engine_network_gas_dispatchable.output (MW),agriculture_chp_engine_network_gas_must_run.output (MW),agriculture_chp_wood_pellets.output (MW),agriculture_flexibility_p2h_hydrogen_electricity.output (MW),agriculture_flexibility_p2h_network_gas_electricity.output (MW),buildings_solar_pv_solar_radiation.output (MW),energy_battery_solar_electricity.output (MW),energy_battery_wind_electricity.output (MW),energy_chp_coal_gas.output (MW),...,transport_motorcycle_using_electricity.input (MW),transport_passenger_train_using_electricity.input (MW),transport_plane_using_electricity.input (MW),transport_ship_using_electricity.input (MW),transport_tram_using_electricity.input (MW),transport_truck_flexibility_p2p_electricity.input (MW),transport_truck_using_electricity.input (MW),transport_van_flexibility_p2p_electricity.input (MW),transport_van_using_electricity.input (MW),deficit
2030-01-01 00:00,45.0553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.7876,...,1.3513,58.8776,0.0,0.0,1.5660,0.0,3.4697,0.0,18.0699,0.0
2030-01-01 01:00,40.1022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.7876,...,0.5032,21.9227,0.0,0.0,0.5831,0.0,2.6220,0.0,13.6555,0.0
2030-01-01 02:00,42.9662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.7876,...,0.2369,10.3222,0.0,0.0,0.2745,0.0,2.0581,0.0,10.7184,0.0
2030-01-01 03:00,38.1514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.7876,...,0.1647,7.1757,0.0,0.0,0.1909,0.0,1.6233,0.0,8.4538,0.0
2030-01-01 04:00,40.9550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.7876,...,0.1648,7.1789,0.0,0.0,0.1909,0.0,1.2742,0.0,6.6357,0.0


In [15]:
# load example mappping
mapping = pd.read_csv('data/hourly_electricity_curve_mapping.csv', index_col=0)
mapping.head()

,name
key,
agriculture_chp_engine_biogas.output (MW),flexibility
agriculture_chp_engine_network_gas_dispatchable.output (MW),flexibility
agriculture_chp_engine_network_gas_must_run.output (MW),flexibility
agriculture_chp_wood_pellets.output (MW),flexibility
agriculture_flexibility_p2h_hydrogen_electricity.input (MW),flexibility


In [3]:
from pyetm.utils import categorise_curves

# categorize electricity curves
mapped = categorise_curves(ecurves, mapping)
mapped.head()

name,deficit,demand,exchange,flexibility,supply
2030-01-01 00:00,0.0,-12672.3245,-5461.1965,15243.3865,2890.1346
2030-01-01 01:00,0.0,-12367.5268,-5816.9354,15238.4334,2946.0288
2030-01-01 02:00,0.0,-11920.4062,-5850.0000,14636.5911,3133.8153
2030-01-01 03:00,0.0,-11559.1973,-5850.0000,14071.9090,3337.2883
2030-01-01 04:00,0.0,-11423.5429,-5850.0000,13823.1756,3450.3669


Alternativly the mapping can also be applied directly from the client.

In [16]:
mapped = client.categorise_curves('electricity', mapping)
mapped.head()

name,deficit,demand,exchange,flexibility,supply
2030-01-01 00:00,0.0,-12672.3245,-5461.1965,15243.3865,2890.1346
2030-01-01 01:00,0.0,-12367.5268,-5816.9354,15238.4334,2946.0288
2030-01-01 02:00,0.0,-11920.4062,-5850.0000,14636.5911,3133.8153
2030-01-01 03:00,0.0,-11559.1973,-5850.0000,14071.9090,3337.2883
2030-01-01 04:00,0.0,-11423.5429,-5850.0000,13823.1756,3450.3669


**Regionalisation**

Curves can also be regionionalized by distributing them over different regions/nodes.

In [18]:
import numpy as np

# this example uses a randomly generated regionalisation
# the regionalization is compatible with the 'mapped' electricity curves.
values = np.random.dirichlet(np.ones(10), size=len(mapped.columns))
reg = pd.DataFrame(values.T, columns=mapped.columns)

reg.round(4).head()

name,deficit,demand,exchange,flexibility,supply
0,0.4448,0.0825,0.0520,0.0276,0.2685
1,0.0259,0.0098,0.0546,0.0296,0.0128
2,0.2500,0.0381,0.2170,0.0365,0.0112
3,0.0774,0.1078,0.0435,0.0316,0.0245
4,0.0224,0.1581,0.3292,0.1006,0.0888


In [19]:
from pyetm.utils import regionalise_curves

# regionalize the mapped curves
# the regionalisation returns the residual profile
regionalised = regionalise_curves(mapped, reg=reg)
regionalised.round(4).head()

,0,1,2,3,4,5,6,7,8,9
2030-01-01 00:00,-133.6507,66.1195,-1079.4716,-1051.6639,-2011.1177,1950.9637,1390.8179,-465.6299,57.3226,1276.3103
2030-01-01 01:00,-112.1409,50.2573,-1144.5996,-1033.0525,-2075.5879,1975.4967,1365.7878,-361.7210,61.0851,1274.4750
2030-01-01 02:00,-43.1303,37.4374,-1154.5838,-1000.6926,-2059.6620,1922.2889,1326.3296,-353.1303,124.3420,1200.8013
2030-01-01 03:00,25.7483,26.8831,-1159.1361,-974.6015,-2041.3003,1870.9578,1295.4370,-364.1377,191.0867,1129.0625
2030-01-01 04:00,60.4505,22.3065,-1161.7721,-965.0637,-2034.8372,1848.9117,1284.7389,-376.3132,226.1083,1095.4699


The `regionalise_curves` function returns the residual profiles per region/node.
This is mainly due to performance and memory constraints that make it difficult 
to hold the mapped curves for each region/node in memory. When needed, the 
`regionalise_node` function makes it possible to return the detailed profiles
for the selected node.

In [11]:
from pyetm.utils import regionalise_node

# return detailed curves for a specific node
ncurves = regionalise_node(mapped, reg=reg, node=0)
ncurves.round(4).head()

name,deficit,demand,exchange,flexibility,supply
2030-01-01 00:00,0.0,-10.0698,-246.1838,1701.2151,47.9603
2030-01-01 01:00,0.0,-9.8276,-262.2200,1700.6623,48.8878
2030-01-01 02:00,0.0,-9.4723,-263.7106,1633.4946,52.0040
2030-01-01 03:00,0.0,-9.1853,-263.7106,1570.4741,55.3806
2030-01-01 04:00,0.0,-9.0775,-263.7106,1542.7146,57.2571
